# AllenNLP tests

Use AllenNLP's semantic role labeling in combination with manually written rules for extracting causal relations from political speeches. We found two limitations: only relations within a single sentence were found and complex relations involving understanding the sentence were hard to identify.

Links:

* software installation: https://github.com/allenai/allennlp (do not forget to install NLTK popular models)
* software usage: https://demo.allennlp.org/semantic-role-labeling (tab: Model Usage)

In [13]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import re
import spacy

In [23]:
spacy_analyze = spacy.load('en_core_web_sm')

In [2]:
srl_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

/home/erikt/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/tango/__init__.py:18: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  "AllenNLP Tango is an experimental API and parts of it might change or disappear "
2021-11-25 16:35:52,066 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2021-11-25 16:35:52,338 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz is up-to-date
2021-11-25 16:35:52,340 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz from cache at /home/erikt/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1cf4758dcee2bb3
2021-11-25 16:35:52,341 - INFO - allennlp.models.archival - extractin

In [3]:
COLOR_CONCEPT_1 = 1
COLOR_CONCEPT_2 = 4
COLOR_CONTENT_RELATION_EXPLANATION = 0
NO_COLOR = -1
RELATION_VERBS = [ "achieve", "allow", "avoid", "balance", "boost", "bring", "brought", "consolidate", "compromise",  "contain", "contribute", 
                   "eliminate", "endanger", "entail", "ensure", "erode", "fail", "flow", "gave", "give", "given", "increase", "open", "preserve", "protect", 
                   "reduce", "reinforce", "restore", "result", "safeguard", "secure", "strenghten", "support", "tackle", "trigger", "undermine", "weaken", ]
REVERSE_VERBS = [ "arise", "need" ]

In [4]:
def strip_tag(token):
    if len(token) > 1:
        if re.search("^\[", token):
            token = re.sub("^\[", "", token)
            token = re.sub(":$", "", token)
        token = re.sub("\]$", "", token)
    return token

In [109]:
def print_with_color(string, color_code):
    print(f"\x1b[1;3{color_code};47m{string}\x1b[m", end="")

In [114]:
def print_argm_prp(token, argument, first_token, last_token):
    if argument == "ARGM-PRP":
        color_code = COLOR_CONCEPT_2
    elif argument in ["V", "ARG0", "ARG1", "ARG2" ]:
        color_code = COLOR_CONCEPT_1
    else:
        color_code = NO_COLOR
    if color_code == NO_COLOR:
        print_token(token, argument, first_token, last_token)
    else:
        print_token_color(token, argument, first_token, last_token, color_code)

In [124]:
def print_argm_dis(token, argument, first_token, last_token):
    color_code = NO_COLOR
    if argument == "ARGM-DIS":
        color_code = COLOR_CONTENT_RELATION_EXPLANATION
    elif arguments != "":
        color_code = COLOR_CONCEPT_2
    if color_code == NO_COLOR:
        print_token(token, argument, first_token, last_token)
    else:
        print_token_color(token, argument, first_token, last_token, color_code)

In [133]:
def print_v(token, argument, first_token, last_token):
    if argument == "V":
        color_code = COLOR_CONTENT_RELATION_EXPLANATION
    elif argument == "ARG0":
        color_code = COLOR_CONCEPT_1
    elif argument in [ "ARG1", "ARG2" ]:
        color_code = COLOR_CONCEPT_2
    else:
        color_code = NO_COLOR
    if color_code == NO_COLOR:
        print_token(token, argument, first_token, last_token)
    else:
        print_token_color(token, argument, first_token, last_token, color_code)

In [123]:
def print_reverse_v(token, argument, first_token, last_token):
    if argument == "V":
        color_code = COLOR_CONTENT_RELATION_EXPLANATION
    elif argument == "ARG0":
        color_code = COLOR_CONCEPT_2
    elif argument in [ "ARG1", "ARG2" ]:
        color_code = COLOR_CONCEPT_1
    else:
        color_code = NO_COLOR
    if color_code == NO_COLOR:
        print_token(token, argument, first_token, last_token)
    else:
        print_token_color(token, argument, first_token, last_token, color_code)

In [10]:
def check_verb(verb, RELATION_VERBS):
    if verb in RELATION_VERBS:
        return True
    if re.sub("[ds]$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ed$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ing$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ing$", "e", verb) in RELATION_VERBS:
        return True
    return False

In [104]:
def print_token(token, argument, first_token, last_token):
    if first_token:
        print(f"[{argument} ", end="")
    print(f"{token}", end="")
    if last_token:
        print(f"]", end="")
    print(" ", end="")

In [108]:
def print_token_color(token, argument, first_token, last_token, color_code):
    if first_token:
        print_with_color(f"[{argument} ", color_code)
    print_with_color(f"{token}", color_code)
    if last_token:
        print_with_color(f"]", color_code)
        print(" ", end="")
    else:
        print_with_color(" ", color_code)

In [129]:
def find_verb(tokens, arguments):
    verb = ""
    for i in range(0, len(arguments)):
        if arguments[i] == "V":
            verb = tokens[i]
    return verb

In [130]:
def pretty_print(analyzed_sentence, lemmas):
    tokens, arguments, first_tokens, last_tokens = convert_analysis(analyzed_sentence)
    for i in range(0, len(tokens)):
        if "ARGM-PRP" in arguments:
            print_argm_prp(tokens[i], arguments[i], first_tokens[i], last_tokens[i])
        elif "ARGM-DIS" in arguments:
            print_argm_dis(tokens[i], arguments[i], first_tokens[i], last_tokens[i])
        elif "V" in arguments and check_verb(find_verb(tokens, arguments), RELATION_VERBS):
            print_v(tokens[i], arguments[i], first_tokens[i], last_tokens[i])
        elif "V" in arguments and check_verb(find_verb(tokens, arguments), REVERSE_VERBS):
            print_reverse_v(tokens[i], arguments[i], first_tokens[i], last_tokens[i])
        else:
            print_token(tokens[i], arguments[i], first_tokens[i], last_tokens[i])
    print("")

In [56]:
def convert_analysis(analyzed_sentence):
    tokens_in = analyzed_sentence.split()
    arguments = []
    tokens_out = []
    first_tokens = []
    last_tokens = []
    current_argument = ""
    first_token = False
    for i in range(0, len(tokens_in)):
        if re.search("^\[", tokens_in[i]):
            current_argument = strip_tag(tokens_in[i])
            first_token = True
            continue
        arguments.append(current_argument)
        tokens_out.append(strip_tag(tokens_in[i]))
        first_tokens.append(first_token)
        last_tokens.append(False)
        if re.search("\]$", tokens_in[i]):
            current_argument = ""
            last_tokens[-1] = True
        first_token = False
    return [tokens_out, arguments, first_tokens, last_tokens]

In [40]:
def analyze(sentence):
    srl_analysis = srl_predictor.predict(sentence=sentence)
    lemmas = get_lemmas(re.sub("\s+", " ", sentence))
    for verb_data in srl_analysis['verbs']:
        pretty_print(verb_data['description'], lemmas)

In [26]:
def get_lemmas(sentence):
    lemmas = []
    results = spacy_analyze(sentence)
    for token in results:
        lemmas.append(token.lemma_)
    return lemmas

In [131]:
analyze("John sees the mountain with the snow")

[ARG0 John] [V sees] [ARG1 the mountain with the snow] 


In [134]:
analyze("""You, in Greece, with our support, need to rebuild your country, your structures, your administration, 
           your economy to increase the competitiveness of Greece.""")

[ARG0 You] , [ARGM-LOC in Greece] , [ARGM-MNR with our support] , [V need] [ARG1 to rebuild your country , your structures , your administration , your economy to increase the competitiveness of Greece] . 
[ARG0 You] , in Greece , with our support , need to [V rebuild] [ARG1 your country , your structures , your administration , your economy] [ARGM-PRP to increase the competitiveness of Greece] . 
You , in Greece , with our support , need to rebuild your country , your structures , your administration , your economy to [V increase] [ARG1 the competitiveness of Greece] . 


In [15]:
analyze("This opened the way to welfare gains from stronger economic and financial integration.")

[ARG0: This] [V: opened] [ARG1: the way to welfare gains from stronger economic and financial integration] . 


In [16]:
analyze("One risk is the temptation for governments to overborrow because the economic costs of excessive public debt")

[ARG1: One risk] [V: is] [ARG2: the temptation for governments to overborrow because the economic costs of excessive public debt] 


In [17]:
analyze("A resulting loosening of fiscal discipline in individual member states can endanger the stability-oriented monetary policy.")

A [V: resulting] [ARG2: loosening] of fiscal discipline in individual member states can endanger the stability - oriented monetary policy . 
A resulting loosening of fiscal discipline in individual member states [V: can] endanger the stability - oriented monetary policy . 
[ARG0: A resulting loosening of fiscal discipline in individual member states] [ARGM-MOD: can] [V: endanger] [ARG1: the stability - oriented monetary policy] . 
A resulting loosening of fiscal discipline in individual member states can endanger the [ARGM-MNR: stability] - [V: oriented] [ARG1: monetary policy] . 


In [18]:
analyze("Therefore, the markets did not properly perform their expected policing function.")

Therefore , the markets [V: did] not properly perform their expected policing function . 
[ARGM-DIS: Therefore] , [ARG0: the markets] did [ARGM-NEG: not] [ARGM-MNR: properly] [V: perform] [ARG1: their expected policing function] . 
Therefore , the markets did not properly perform their [V: expected] [ARG1: policing function] . 


In [19]:
analyze("I hope that following the lessons of interdependence not only at global, but also at European level given by the crisis")

[ARG0: I] [V: hope] [ARG1: that following the lessons of interdependence not only at global , but also at European level given by the crisis] 
I hope that [V: following] [ARG1: the lessons of interdependence] [ARGM-LOC: not only at global , but also at European level given by the crisis] 
I hope that following [ARG1: the lessons of interdependence] not only at global , but also at European level [V: given] [ARG0: by the crisis] 


In [20]:
analyze("European economic strategy needs the full commitment of the European political community.")

[ARG0: European economic strategy] [V: needs] [ARG1: the full commitment of the European political community] . 


In [25]:
analyze("And a European economic system whose resilience flows from its single market")

And a European economic system [ARG1: whose resilience] [V: flows] [ARGM-DIR: from its single market] 


In [26]:
analyze("there are of course considerable budgetary challenges arising from the recent exceptional measures")

there [V: are] [ARGM-DIS: of course] [ARG1: considerable budgetary challenges arising from the recent exceptional measures] 
there are of course [ARG1: considerable budgetary challenges] [V: arising] [ARG2: from the recent exceptional measures] 


In [ ]:
analyze("And the best hope of a return to growth and job creation is inside the euro area.")

In [ ]:
analyze("To conclude, let me say a few words on the euro area more generally.")

In [ ]:
analyze("""We have taken important, fundamental decisions over the last couple of months to safeguard the stability 
           of the euro area, and indeed we are now in the phase of implementation.""")

In [ ]:
analyze("""A number of governments have embarked on a path of reform and fiscal consolidation that was unthinkable only 
           very recently, and they have taken important decisions and I encourage them to keep this determination.""")

In [ ]:
analyze("""These reforms are now being implemented and this effort must continue with credibility, with consistency, 
           with coherence over time.""")

In [ ]:
analyze("As we said there will not be magic solutions.")

In [ ]:
analyze("We need sustained efforts and determination.")

In [ ]:
analyze("At the same time, the existing financial backstops are being used as necessary.")

In [ ]:
analyze("""Most recently, the financial assistance to the recapitalisation of Spanish banks has been agreed 
           and is ready for implementation.""")

In [ ]:
analyze("""Giving to the ECB the ultimate responsibility for supervision of banks in the euro area 
           will decisively contribute to increase confidence between the banks 
           and in this way increase the financial stability in the euro area.""")

In [ ]:
analyze("Second, Germany is acutely aware of the need to tackle the root causes and not just the symptoms of the crisis.")

In [ ]:
analyze("""This is why it is pressing strongly for institutional reforms of the EMU framework plus structural reforms 
           and budgetary discipline in the member states.""")

In [ ]:
analyze("The introduction of the euro eliminated exchange rate risks.")

In [ ]:
analyze("""Another implication of the euro area's single monetary policy is that the key interest rates 
           are set for the currency bloc as a whole.""")

In [ ]:
analyze("""One risk is the temptation for governments to overborrow because the economic costs of excessive public debt, 
           for example higher interest rates, can be more easily shifted to other member states.""")

In [ ]:
analyze("""Even Germany ran up excessive deficits for a few years and, even worse, championed a reform of the SGP 
           which ultimately further weakened the application of the fiscal rules.""")

In [ ]:
analyze("""However, the EMU framework not only failed to avoid excessive deficits, it was also unable to prevent 
           the build-up of macroeconomic imbalances within the euro area.""")

In [ ]:
analyze("""The resulting increase in domestic inflation and wages eroded the competitiveness of the countries concerned and 
           increased their dependence on capital imports.""")

In [ ]:
analyze("The task of implementing the reforms and regaining competitiveness entailed significant political and social costs.")

In [ ]:
analyze("""However, these efforts, supported by a strong expansion in the global economy, allowed German growth to rebound 
           after 2005.""")

In [ ]:
analyze("""In order to achieve a turnaround and allow further assistance, it is now essential for Greece to deliver 
           on the promises that have been made.""")

Modifiers in Propbank (source: http://clear.colorado.edu/compsem/documents/propbank_guidelines.pdf )
* ADJ: Adjectival
* ADV: Adverbials
* CAU: Cause
* COM: Comitative
* DIR: Directional
* DIS: Discourse
* DSP: Direct Speech
* EXT: Extent
* GOL: Goal
* LOC: Locative
* LVB: Light Verb
* MNR: Manner
* MOD: Modal
* NEG: Negation
* PRD: Secondary Predication
* PRP: Purpose
* REC: Reciprocals
* SLC: Relative Clause